In [4]:
import os
import re
import csv
import pickle
import googleapiclient.errors
import google_auth_oauthlib.flow
import googleapiclient.discovery

api_version = "v3"
api_service_name = "youtube"
client_secrets_file = "client_secrets.json"
scopes = ["https://www.googleapis.com/auth/youtube.readonly"]
filters = ['vk','music','instagram','tiktok']

In [25]:
def get_auth_service():
    cpf = "DELETE_FILE"
    # cpf = "CREDENTIALS_PICKLE_FILE"
    if os.path.exists(cpf):
        with open(cpf, 'rb') as f:
            credentials = pickle.load(f)
    else:
        flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(client_secrets_file, scopes)
        auth_url = flow.authorization_url()
        print("URL IS: ", auth_url)
        #credentials = flow.run_console()
        credentials = flow.run_local_server(host='localhost',
                                            port=6969, 
                                            authorization_prompt_message='Please visit this URL: {url}', 
                                            success_message='The auth flow is complete; you may close this window.',
                                            open_browser=True)
        with open(cpf, 'wb') as f:
            pickle.dump(credentials, f)
    return googleapiclient.discovery.build(api_service_name, api_version, credentials = credentials)

# def get_auth_service():
#     cpf = "CREDENTIAL_PICKLE_FILE"
#     # cpf = "CREDENTIALS_PICKLE_FILE"
#     if os.path.exists(cpf):
#         with open(cpf, 'rb') as f:
#             credentials = pickle.load(f)
#     else:
#         flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(client_secrets_file, scopes)
#         auth_url = google_auth_oauthlib.flow.authorization_url
#         credentials = flow.run_console()
#         with open(cpf, 'wb') as f:
#             pickle.dump(credentials, f)
#     return googleapiclient.discovery.build(api_service_name, api_version, credentials = credentials)

In [26]:
def get_latest_video_description(channel_id):
    youtube = get_auth_service()

    response = youtube.channels().list(
        part="snippet,contentDetails",
        id=channel_id
    ).execute()

    # get "uploads" playlist ID
    channel = response['items'][0]
    contentDetails = channel['contentDetails']
    relatedPlaylists = contentDetails['relatedPlaylists']
    uploads = relatedPlaylists['uploads']
    
    playlist = youtube.playlistItems().list(
        part="snippet,contentDetails",
        maxResults=1,
        playlistId=uploads
    ).execute()

    videos = playlist['items']
    latest_video = videos[0]
    latest_video_description = latest_video['snippet']['description']
    return latest_video_description



get_latest_video_description("UC-lHJZR3Gqxm24_Vd_AJ5Yw")

URL IS:  ('https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=598157537762-pc497p39hoqoae4pg70hqcl6qi7obltv.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.readonly&state=Tt5KpCJWOOlqwcRBN5mSw7bxBB5IIy&access_type=offline', 'Tt5KpCJWOOlqwcRBN5mSw7bxBB5IIy')
Please visit this URL: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=598157537762-pc497p39hoqoae4pg70hqcl6qi7obltv.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A6969%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.readonly&state=bRwFEqGUlVx8Yl0OLp2OgM1ommyPcf&access_type=offline


"🧎Subscribe to become a FLOOR GANG Member here: 👉https://www.youtube.com/channel/UC-lHJZR3Gqxm24_Vd_AJ5Yw/join\nGet exclusive epic pewdiepie inside epic access and large 🅿️🅿️!\n\n🥤PewDiePie Gfuel(affiliate link): https://gfuel.ly/31Kargr\n#CodePewdiepie\n\n✨My Stores✨\n👕 Merch: https://represent.com/store/pewdiepie\n👘 Tsuki: https://tsuki.market/\n👔 Based: https://www.based.gg\n🗿 100M Figurine: https://pewdiepie.store/\n📱Customized Devices: https://rhinoshield.io/pewdiepie\n👕 Terraria Collab: https://terraria.shop/collections/pewdiepie\n\n\n⚙️My Setup (affiliate link)⚙️\n🪑 Chair: https://clutchchairz.com/pewdiepie/\n⌨️ Keyboard: https://ghostkeyboards.com/pages/pewdiepie\n🖱️ Mouse: https://ghostkeyboards.com/pages/pewdiepie\n\n🕹️ Pewdiepie's Pixelings\niOS: https://buff.ly/2pNG0aT\nAndroid: https://buff.ly/34C68nZ\n#pewdiepie #pixelings\n\n🕹️Pewdiepie’s Tuber Simulator\niOS: https://apps.apple.com/us/app/pewdiepies-tuber-simulator/id1093190533\nAndroid: https://play.google.com/store/ap

In [7]:
def get_links(description):
    return re.findall("(?P<url>https?://[^\s]+)", description)

In [8]:
def filter_links(links):
    return [link for link in links if not re.search("|".join(["("+f+")" for f in filters]),link)]

In [9]:
def parse():
    opened_file = open('input.csv', encoding="UTF-8")
    reader = csv.reader(opened_file)

    for row in reader: 
        try:
            channel_id = row[2]
            description = get_latest_video_description(channel_id)
            links = get_links(description)
            filtered_links = filter_links(links)
            print(filtered_links)
        except:
            print(reader.line_num)

In [10]:
# this method allows developer to specify the number of videos to get links from
def get_links_from_videos(channel_id,number_of_videos=1):
    youtube = get_auth_service()

    response = youtube.channels().list(
        part="snippet,contentDetails",
        id=channel_id
    ).execute()

    # get "uploads" playlist ID
    channel = response['items'][0]
    contentDetails = channel['contentDetails']
    relatedPlaylists = contentDetails['relatedPlaylists']
    uploads = relatedPlaylists['uploads']
    
    playlist = youtube.playlistItems().list(
        part="snippet,contentDetails",
        maxResults=number_of_videos,
        playlistId=uploads
    ).execute()

    videos = playlist['items']
    all_links = []
    for i in range(number_of_videos):
        video_description = videos[i]['snippet']['description']
        links = get_links(video_description)
        filtered_links = filter_links(links)
        all_links += filtered_links
    #here I return unique links
    return list(dict.fromkeys(all_links))

In [11]:
pewds = 'UC-lHJZR3Gqxm24_Vd_AJ5Yw'
links = get_links_from_videos(pewds,10)

print(links)

['https://www.youtube.com/channel/UC-lHJZR3Gqxm24_Vd_AJ5Yw/join', 'https://gfuel.ly/31Kargr', 'https://represent.com/store/pewdiepie', 'https://tsuki.market/', 'https://www.based.gg', 'https://pewdiepie.store/', 'https://rhinoshield.io/pewdiepie', 'https://terraria.shop/collections/pewdiepie', 'https://clutchchairz.com/pewdiepie/', 'https://ghostkeyboards.com/pages/pewdiepie', 'https://buff.ly/2pNG0aT', 'https://buff.ly/34C68nZ', 'https://apps.apple.com/us/app/pewdiepies-tuber-simulator/id1093190533', 'https://play.google.com/store/apps/details?id=com.outerminds.tubular&hl=en_GB&gl=US', 'https://youtu.be/FQgLsYOKP8w', 'https://NordVPN.com/pewdiepie', 'https://www.youtube.com/watch?v=CD7UmmLhFR8', 'https://store.steampowered.com/app/979070/Just_Die_Already/', 'https://www.youtube.com/channel/UCepq9z9ovYGxhNrvf6VMSjg', 'https://gfuel.ly/pewdiepie']


In [12]:
def perform(func):
    def inner():
        x = 6
        y = 9
        func(x,y)
    return inner

@perform
def add(x,y):
    z = x+y
    print(z)

@perform 
def mult(x,y):
    z = x*y
    print(z)

add()
mult()

15
54
